In [34]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [35]:
from Discriminator import Discriminator
from Generator import Generator
import torchvision.datasets
import torch.utils.data

In [36]:
ngpu = torch.cuda.device_count()
batch_size = 128
nc = 3
ngf = 64
ndf = 64
lr = 0.0002
beta1 = 0.5
image_size = 64
nz = 100
num_epochs = 5
num_workers = 2

In [37]:
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = dset.ImageFolder(root='./data/',
                           transform=transforms)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [38]:
def init_weights(m):
    if isinstance(m, (nn.ConvTranspose2d, nn.Conv2d)):
        nn.init.normal_(m.weight.data, 0, 0.2)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [41]:
generator = Generator(nz, ngf, nc, ngpu)

generator.apply(init_weights)

discriminator = Discriminator(ngpu, nc, ndf)

discriminator.apply(init_weights)


HELLOOO


Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.2, inplace=True)
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
    (13): Sigmoid()
  )
)

In [43]:
criterion = nn.BCELoss()
G_opt = optim.Adam(params=generator.parameters(), lr=lr, betas=(0.5, 0.999))
D_opt = optim.Adam(params=discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
real_label = 1
fake_label = 0

num_epochs = 1

discriminator_losses = []
generator_losses = []
iters = 0
grids = []

fake_noise = torch.randn(64, 100, 1, 1)

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        #TRAIN DISCRIMINATOR
        # run real images through discriminator
        images, _ = data
        D_opt.zero_grad()
        output = discriminator(images).view(-1, 1)
        labels_real = torch.empty(batch_size, 1).fill_(1)
        dreal_loss = criterion(output, labels_real)
        dreal_loss.backward()

        # run fake images through discriminator
        start_vectors = torch.randn(batch_size, 100, 1, 1)
        g_output = generator(start_vectors)
        d_output = discriminator(g_output.detach()).view(-1, 1)
        labels_fake = torch.empty(batch_size, 1).fill_(0)
        dfake_loss = criterion(d_output, labels_fake)
        dfake_loss.backward()

        # optimize discriminator
        dloss = dfake_loss.item() + dreal_loss.item()
        discriminator_losses.append(dloss)
        D_opt.step()

        #TRAIN GENERATOR
        G_opt.zero_grad()
        generated_images = generator(start_vectors)
        d_generated_images = discriminator(generated_images).view(-1, 1)
        dgen_loss = criterion(d_generated_images, labels_real)
        generator_losses.append(dgen_loss.item())
        dgen_loss.backward()
        G_opt.step()
        if (i == 1000):
            break
        if (i % 50 == 0):
            print(f'EPOCH: {epoch}, i: {i}, Discriminator fake loss: {dfake_loss.item()}, Discriminator real loss: {dreal_loss.item()}, Discriminator loss: {dloss},Generator loss: {dgen_loss.item()}')
        
        

        if (iters % 500 == 0):
            with torch.no_grad():
                output = generator(fake_noise).detach()
                grids.append(output)
        
        iters += 1




KeyboardInterrupt: 

In [ ]:
torch.save(discriminator.state_dict(), './Models/discriminator')
torch.save(generator.state_dict(), './Models/generator')

In [32]:
print(generator_losses)
print(iters)

[9.46702766418457, 8.30473518371582, 7.0860066413879395, 7.626431465148926, 8.628865242004395, 12.746368408203125, 10.48304557800293, 11.772764205932617, 10.355409622192383, 11.631795883178711, 12.126100540161133, 13.686363220214844, 11.79005241394043, 11.922568321228027, 11.632925033569336, 12.964308738708496, 12.188389778137207, 13.441307067871094, 13.150005340576172, 13.802115440368652, 13.384523391723633, 13.742122650146484, 14.775303840637207, 15.30206298828125, 15.002272605895996, 14.009035110473633, 14.374255180358887, 14.634359359741211, 15.597799301147461, 15.001480102539062, 13.250810623168945, 14.399550437927246, 13.954501152038574, 14.65098762512207, 15.759428024291992, 17.343778610229492, 15.253091812133789, 17.072498321533203, 15.972383499145508, 16.089431762695312, 16.176597595214844, 16.5985107421875, 15.97970962524414, 14.983637809753418, 15.772771835327148, 16.15467071533203, 16.21337890625, 16.669361114501953, 18.57256507873535, 16.69689178466797, 16.049760818481445,

In [44]:
len(dataloader)

1583